In [1]:
from openai import AzureOpenAI
import json
from os import environ
import os
import openai


In [3]:
# client = openai.OpenAI(
#     api_key=environ["OPENAI_API_KEY"],
#     base_url=environ["OPENAI_ENDPOINT"]
# )



In [4]:
models = client.models.list()
print(models)

SyncPage[Model](data=[Model(id='openai.text-embedding-3-large', created=1677610602, object='model', owned_by='openai'), Model(id='cohere.command-r-plus', created=1677610602, object='model', owned_by='openai'), Model(id='cohere.embed-english.v3', created=1677610602, object='model', owned_by='openai'), Model(id='cohere.command-r', created=1677610602, object='model', owned_by='openai'), Model(id='meta.llama-3.2-11b-vision-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='openai.gpt-4o', created=1677610602, object='model', owned_by='openai'), Model(id='amazon.titan-text-embeddings.v2', created=1677610602, object='model', owned_by='openai'), Model(id='openai.gpt-4o-mini', created=1677610602, object='model', owned_by='openai'), Model(id='meta.llama-3.2-90b-vision-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='meta.llama-3.1-405b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='openai.gpt-4o-mini.2024-07-18',

In [5]:
response = client.chat.completions.create(
    model="openai.gpt-4o-mini",
    messages=[
        {"role": "user", "content": "this is a test request, write a short poem"}
    ]
)
print(response.choices[0].message.content)

In twilight's whisper, shadows play,  
Stars emerge to greet the day,  
Moonbeams dance on silver streams,  
Cradling softly our sweetest dreams.  

Breezes carry secrets light,  
As fireflies blink in the night,  
Nature's lullaby, pure and true,  
A symphony for me and you.  


In [6]:
! pip install transformers
! pip install torch
! pip install einops timm
! pip install datasets
! pip install librosa
! pip install soundfile
! pip install accelerate
! pip install gtts

from transformers import AutoProcessor, AutoModelForCausalLM  
from PIL import Image
from os import environ
from io import BytesIO
import os
import requests
import copy
import torch
import base64


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Initialize conversation history
conversation_history = [
    {
        "role": "system",
        "content": """
        You are a Retrieval-Augmented Visual Memory Assistant called DejaVu designed to help memory-impaired individuals 
        enrich their memories and recall details about their experiences. 

        Your goal is to engage the user in a **natural, conversational manner**. Do not provide lists, 
        numbered responses, or multiple suggestions in one reply. Keep your answers **clear, concise, and engaging**.

        Start by analyzing the provided image description. Summarize it in one positive, friendly sentence, 
        then follow up with a single conversational question to encourage the user to share more details. 
        Focus on personal experiences, emotions, and meaningful moments.

        Conclude the conversation when you have gathered enough meaningful context by:
        - Thanking the user for sharing their memories.
        - Asking if there is anything else they would like to add.
        - Politely signaling the end of the memory-annotation process if the user has no additional input.

        Example:
        Input: "A group of people sitting on the grass in front of a large brick building."
        Output: "This looks like such a cheerful day with friends! Can you share where this photo was taken?"

        Additional Instructions:
        - Use empathetic and supportive language.
        - Avoid repeating questions or providing overly lengthy responses.
        - If the user struggles to recall details, gently encourage them without pressure.
        """
    }
]

# Define a function to initialize the conversation with an image description
def initialize_conversation(image_description):
    """
    Starts the conversation based on an image description by generating a friendly opening line.
    """
    try:
        # Add the image description as the initial user input
        conversation_history.append({"role": "user", "content": f"Image description: {image_description}"})
        
        # Make the API call to generate the assistant's initial response
        response = client.chat.completions.create(
            model="openai.gpt-4o-mini",  # Replace with the desired model name
            messages=conversation_history,
            max_tokens=150,  # Adjust as needed
            temperature=0.5  # Lower temperature for more focused output
        )
        
        # Get the assistant's response
        assistant_response = response.choices[0].message.content.strip()
        
        # Add the assistant's response to the conversation history
        conversation_history.append({"role": "assistant", "content": assistant_response})
        
        return assistant_response
    except Exception as e:
        return f"An error occurred: {e}"

# Define a function to continue the chat with the assistant
def chat_with_assistant(user_input):
    """
    Sends user input to the OpenAI chat model and returns the assistant's response.
    """
    try:
        # Add the user input to the conversation history
        conversation_history.append({"role": "user", "content": user_input})
        
        # Make the API call
        response = client.chat.completions.create(
            model="openai.gpt-4o-mini",  # Replace with the desired model name
            messages=conversation_history,
            max_tokens=300,  # Adjust as needed
            temperature=0.5  # Lower temperature for more consistent responses
        )
        
        # Get the assistant's response
        assistant_response = response.choices[0].message.content.strip()
        
        # Add the assistant's response to the conversation history
        conversation_history.append({"role": "assistant", "content": assistant_response})
        
        return assistant_response
    except Exception as e:
        return f"An error occurred: {e}"

# Start the chatbot loop
def run_chatbot(image_description):
    # Initialize the conversation with the image description
    initial_response = initialize_conversation(image_description)
    print(f"Assistant: {initial_response}")
    
    interaction_count = 0
    max_interactions = 5  # Define the threshold for interactions
    
    print("\nChatbot is ready! Type 'exit' to end the conversation.")
    while True:
        # Increment interaction count
        interaction_count += 1
        
        # Check if the conversation should conclude
        if interaction_count > max_interactions:
            print("Assistant: Thank you so much for sharing your memories with me. Is there anything else you'd like to add?")
            user_input = input("You: ")
            
            if user_input.strip().lower() in ["no", "nothing else", "exit"]:
                print("Assistant: I'm glad I could help. Take care and have a wonderful day!")
                break
            else:
                print("Assistant: Thank you for sharing! If you have any more details to add later, feel free to revisit.")
                break
        
        # Get user input
        user_input = input("You: ")
        
        # Exit the loop if the user types 'exit'
        if user_input.lower() == "exit":
            print("Assistant: Thank you for chatting with me. Goodbye!")
            break
        
        # Get the assistant's response
        assistant_response = chat_with_assistant(user_input)
        
        # Print the assistant's response
        print(f"Assistant: {assistant_response}")

# Example usage: Run the chatbot with an image description
example_image_description = """
The image depicts a modern building with a striking design, set against a dramatic sky.

**Main Subject and Setting**
The building's main subject is its unique architecture, featuring a large glass facade and a cantilevered structure that extends over the entrance. The building is situated in a public space, with a sidewalk leading up to it and a grassy area to the right.

**Time of Day and Weather Conditions**
The time of day appears to be dusk, as the sky is overcast and dark clouds are visible. The lighting inside the building suggests that it is illuminated from within, possibly due to the setting sun.

**Colors and Visual Elements**
The building's exterior is primarily composed of glass and metal, with a silver-colored metal cladding that gives it a sleek and contemporary look. The cantilevered structure is supported by thin metal beams, which add to the building's futuristic aesthetic. The sky above is a deep grey, with darker clouds gathering in the distance.

**Emotions and Actions**
There are no visible emotions or actions in the image, as it appears to be a static photograph of the building.

**Significant Objects or Landmarks**
The building itself is the most significant object in the image, with its unique design and architecture making it a notable landmark. In the background, a few trees and other buildings can be seen, but they are not as prominent as the main subject.

**Text and Signs**
The building's name, "Bates Hall," is visible on the front of the structure, just below the cantilevered section. There are no other signs or text visible in the image.

**Clothing and Attire**
There are no people visible in the image, so there is no clothing or attire to describe.

**Surrounding Environment and Background**
The surrounding environment is a public space, with a sidewalk leading up to the building and a grassy area to the right. In the background, a few trees and other buildings can be seen, but they are not as prominent as the main subject.

**Overall Mood and Atmosphere**
"""
run_chatbot(example_image_description)

Assistant: This building, with its striking design and dramatic sky, creates a captivating atmosphere! Have you had any memorable experiences at Bates Hall or nearby?

Chatbot is ready! Type 'exit' to end the conversation.
Assistant: Oh no, I’m sorry to hear that! That must have been quite a painful experience. Can you tell me more about what happened?
Assistant: That sounds really tough, especially with the unexpected nature of it! How did you feel in the moments right after the accident?
Assistant: I can only imagine how difficult that must have been. Did you have someone with you at the time to help you out?
Assistant: That must have felt really overwhelming. It’s tough to go through something like that alone. How did you manage to get the help you needed afterward?
Assistant: That was a smart move. It must have been a relief to have help on the way. How did you feel once the paramedics arrived?
Assistant: Thank you so much for sharing your memories with me. Is there anything else y

In [9]:
from PIL import Image
import base64
import os

# Path to the local image file
local_image_path = "../bill_melinda_gates.jpg"  # Replace with your local file path

# Step 1: Resize the image to fit the model's requirements
def resize_image(image_path, max_size=(512, 512)):
    """
    Resize the image to fit within the max_size while maintaining aspect ratio.
    """
    try:
        with Image.open(image_path) as img:
            img.thumbnail(max_size)  # Resize the image
            resized_path = "resized_image.jpg"  # Temporary file to save the resized image
            img.save(resized_path, format="JPEG")
            print(f"Image resized and saved as '{resized_path}'")
            return resized_path
    except Exception as e:
        print(f"Error resizing image: {e}")
        exit()

# Step 2: Encode the resized image file as Base64
def encode_image_to_base64(image_path):
    """
    Encode the image file as a Base64 string.
    """
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    except FileNotFoundError:
        print(f"Error: File '{image_path}' not found.")
        exit()

# Step 3: Send the Base64 image to the OpenAI endpoint
def send_image_to_openai(base64_image):
    """
    Send the Base64-encoded image to the OpenAI endpoint.
    """
    try:
        full_response = ""
        prompt = [
            {"role": "user", "content": [
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                {"type": "text", "text": """Describe the image in detail, focusing on the following aspects: 
                 1. Main subject(s) or people in the photo, 
                 2. Location or setting, 
                 3. Time of day and weather conditions, 
                 4. Colors and visual elements that stand out, 
                 5. Any visible emotions or actions, 
                 6. Significant objects or landmarks, 
                 7. Any text or signs visible in the image, 
                 8. Clothing or attire of people (if applicable), 
                 9. Surrounding environment or background details, 
                 10. Overall mood or atmosphere of the scene, Please provide a comprehensive description that captures both the visual elements and the context of the image, as if explaining it to someone who cannot see it. 
                 Include any details that might be particularly memorable or emotionally significant."""}
            ]}
        ]
        while True:
            response = client.chat.completions.create(
                model="meta.llama-3.2-11b-vision-instruct",  # Replace with your model name
                messages=prompt,
                max_tokens=500,  # Adjust as needed
                temperature=0.7
            )
            response_content = response.choices[0].message.content
#            print(response_content)
            full_response += response_content
            
            # Check if the response ends abruptly and prompt continuation
            if not response_content.strip().endswith("..."):
                break  # No continuation needed
            prompt = [{"role": "assistant", "content": "Continue describing the image."}]
        
        print("Full Response:", full_response)
    except Exception as e:
        print("Error:", e)

# Workflow
resized_image_path = resize_image(local_image_path, max_size=(512, 512))  # Resize the image
base64_image = encode_image_to_base64(resized_image_path)  # Encode resized image
send_image_to_openai(base64_image)  # Send the image


Image resized and saved as 'resized_image.jpg'
Full Response: The image depicts a modern building, likely a museum or library, with a striking design that features a combination of glass and metal elements. The building's façade is composed of large glass panels and metal accents, which give it a sleek and contemporary appearance.

Here are the key details about the image:

1. **Main subject(s) or people in the photo**: There are no people visible in the image.
2. **Location or setting**: The building appears to be situated in an urban area, possibly on a college campus or in a downtown district. The presence of a road in front of the building suggests that it is located in a populated area.
3. **Time of day and weather conditions**: The sky above the building is cloudy and overcast, indicating that the photo was taken during the late afternoon or early evening. The lighting on the building suggests that the sun is low in the sky, casting a warm glow over the scene.
4. **Colors and vis